In [1]:
import statsmodels.sandbox.distributions.extras as extras
import scipy.interpolate as interpolate
import numpy as np

In [2]:
from statistics import mean
from statistics import variance
from scipy.stats import skew
from scipy.stats import kurtosis
from math import sqrt

In [3]:
def raw_moment(vec, r):
    return 1 / len(vec) * sum(list(map(lambda i: i**r, vec)))
    
def kelly(retvec, int=6):
    moms = []
    for i in range(int):
        moms.append(raw_moment(retvec, i+1))
    p = []
    for i in range(len(moms)):
        p.append((-1)**i * moms[i])
    r = np.polynomial.polynomial.polyroots(p)
    rr = r.real[abs(r.imag)<1e-5]
    return min(rr)

In [4]:
# https://medium.com/@josh_lxdx/kelly-fraction-f-higher-moments-9eedb0adb22c

def kelly_four_moments(mu, sigma, c3, c4):
    return (c3/(3*c4) - (c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)/(3*(-c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/(2*c4**2) + np.sqrt(-4*(c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)**3 
         + (-2*c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/c4**2 - 27*mu/c4)**2)/2 - 27*mu/(2*c4))**(1/3))
         - (-c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/(2*c4**2) + np.sqrt(-4*(c3**2/c4**2 - 3*(mu**2 + sigma**2)/c4)**3
         + (-2*c3**3/c4**3 + 9*c3*(mu**2 + sigma**2)/c4**2 - 27*mu/c4)**2)/2 - 27*mu/(2*c4))**(1/3)/3)

In [5]:
# see https://medium.com/@josh_lxdx/generating-random-variables-with-higher-moments-1bcbd83ec4c8

def generate_normal_four_moments(mu, sigma, skew, kurt, size=1000, sd_wide = 10):
    f = extras.pdf_mvsk([mu, sigma, skew, kurt])
    x = np.linspace(mu - sd_wide * sigma, mu + sd_wide * sigma, num=500)
    y = [f(i) for i in x]
    yy = np.cumsum(y) / np.sum(y)
    inv_cdf = interpolate.interp1d(yy, x, fill_value="extrapolate")
    rr = np.random.rand(size)
    return inv_cdf(rr)

In [6]:
sample = generate_normal_four_moments(.05, .03, .03, -.01)

In [7]:
_mu = mean(sample)
_sigma = sqrt(variance(sample))
_skew = skew(sample)
_kurtosis = kurtosis(sample)

In [8]:
# raw moment kelly
kelly(sample,4) 

1.9907572502568147

In [9]:
# central moment kelly
kelly_four_moments(_mu, _sigma, _skew, _kurtosis)

-0.4263321268463503

In [10]:
# hand coded skew / kurt values
kelly_four_moments(.05, .03, .03, -.01)

-3.5042094115393585

In [11]:
# normal approximation
mean(sample) / variance(sample)

2.079433962824526